<a href="https://colab.research.google.com/github/TouchGrass1/Raceing-Line/blob/main/Nea_pseudo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Find track boundaries

This is the first step taken when a new track is presented. This type of calculation is static and will never change (unless the resolution of the image changes) as a result I can save the final produced array as a numpy binary file to save time having to calculate repeatedly
To find the track boundaries I used a flood fill method, this method works but may not be the most effective as it relies on the back ground being white, the track being black and the fact that there is some from of grey within the image due to "smoothing" features. The algorithm makes its way clockwise around the track finding grey pixels to add to the boundaries.

first the neighbours of the starting pixel (predefined to be blue for outer boundary and green for the inner boundary)

In [ ]:
def get_candidate_neighbors(node, size) #size is specifies the search radius to look for a neighbour
  cx, cy = node
  offset = size // 2
  return [(cx + dx - offset, cy + dy - offset) for dx in range(size) for dy in range(size)] #returns a grid of potential neighbours


In [ ]:
def flood_fill(img_arr, start_blue, start_green):
  sets = [set(), set()]
  sets[0].add((start_blue[0], start_blue[1]))
  sets[1].add((start_green[0], start_green[1]))
  for i, pixel_set in enumerate(sets):
    while pixel_set:
      node = pixel_set.pop()
      img_arr[node] = colours[i]

      for nx, ny in get_candidate_neighbors(node, size=5):
        if x,y within img array:
          if x,y not in set:
            pixel_set.add((int(nx), int(ny)))
    order = find_order(img_arr, start_blue, start_green, colours)


In [ ]:
def find_order(img_arr, start_blue, start_green):
  order = [[start_blue], [start_green]] #does outer then inner boundaries
    for i in range(2):
        run = True
        while run:
            node = order[i][-1] #picks the latest node
            closest = None
            closest_dist = float("inf")

            for size in [3, 5, 11]: #progressivly increase search radius, helps improve speeds
                neighbors = TrackProcessor.get_candidate_neighbors(node, size=size) #find neighbours
                for neighbor in neighbors:
                    if ( neighbour within the bounds of the img_arr):
                        if np.array_equal(img_arr[neighbor[0], neighbor[1]], colours[i]):
                            dist = hypot(node[0] - neighbor[0], node[1] - neighbor[1])
                            if 0 < dist < closest_dist:
                                closest = (neighbor[0], neighbor[1])
                                closest_dist = dist
                                if closest_dist == 1:
                                    break
                if closest is not None:
                    if size == 11:
                        TrackProcessor.join_gap(order[i][-1], closest)
                    break

            if closest is None:
                run = False
            else:
                order[i].append(closest)
  check_return_to_start(order, start_blue, start_green)

In [ ]:
def check_return_to_start(order, start_blue, start_green):
  if hypot(start and last) < root2:
    return True


In [ ]:
def join_gap(node1, node2): #joins any gaps within the finalised boundaries
  draw.line(node1,node2)

# Resample track boundaries and generate mesh

I need to resample the track boundaries because the original track boundaries list is of different lengths because the outer boundary is larger and therefore has more pixels(nodes), however during caluations both lists need to be of the same dimensions

the first method I tried was to find the arc lengths of both boundaries and lerp both of them individually with a fixed amount of points

In [ ]:
def resampleBoundary(order, spacing): #ensures that both boundaries have the same number of points and are evenly spaced
  inner, outer = order #splits the order into inner and outer boundaries
  x,y = inner[:,0], inner[:,1] #splits into x and y coords

  distances = sqrt(diff(x)**2 + diff(y)**2) #distance formula
  sum = cumsum(distances) #cumulative sum, total length

  num_points = sum/spacing #spacing is the distance between each point

  new_x = interpolate(target_distance, sum, x) #linear interpolation of x
  new_y = ... #same as x

however this casued slipage as during bends the inside curve traveled further causing the nodes to get ahead and cause warping to the mesh

so I tried to normalise both arcs and then create the nodes in proportion to the inner arc.

this helped signifcantly and the slipage was reduced however the warping of the mesh was stil noticible

The next method I tried was more mathmatically robust, this method find the normal to the inner arcs' nodes and then calculates the corresponding node on the outer arc

In [ ]:
def unit_tangent_normal(pts)
  m = gradient(pts) #numpy command to find first derivites of the points
  tangent = m/ magnitude(m) #unit tangent given as a 1x2 matrix [x, y]
  normal = ([-T[1], T[0]]) #90deg rotation to find normals

In [ ]:
def line_projection_intersection(inner, outer, normal)# projects a normal line from inner arc to the outer arc
  corresponding_pts = []
  for i, (p, N) in enumerate(zip(normal,inner)): #zip matches inner and its corresponding normal
    best_dist = inf
    best_pt = None # the line will intersect with other points on the outer boundary, only want the closest one
    k = N[0]*p[0] + N[1]*p[1] #line constant
    for pt in outer:
      c = N[0]*pt[0] + N[1]*pt[1]
      if k == c: #point lies on the line
          len = np.hypot(pt[0]-p[0], pt[1]-p[1])
          if len < best_dist:
              best_dist = len
              best_pt = pt
  corresponding_pts.append(best_pt)
  return corresponding_pts


This method allowed me to produce the perfect mesh

However after more thorough research into bezier curves and splines and the sad realization that all the track images I found did not include details on varying track widths (the images depicted that all tracks had a fixed track width), I decided to use the flood fill method to only find the mid line arc of the track and extrolpolate the inner and outer boundaries using a predetermined track width. I was willing to make the trade of losing a higher accuracy of the track itself with a more detailed and higher resolution mesh that is hopefully less computationally expensive because I never could have accessed the track width in the first place.

# Generate track purely through bezier curves

Step 1: ensure image is in grayscale

Step 2: use skimage.skeletonization to extract the centerline

Step 3: simplify path, only want a small sample of points

Step 4: Fit a b-spline to the sampled points (allows for better curvature continuity)

Step 5: find local cooridinates that are ofset from the curve

In [ ]:
img.grayscale
skeleton = skeletonize(img)
epsilon = 5 #space between each sample
approx = appoxPolyDP(skeleton, epsilon, closed = True) #samples but ensures it picks points with more importance

generating b-spline

In [ ]:
def create_b_spline(centerline, degree=3, close=True):
  initialize curve
  set degrees
  autogenerate knot vectors
  evaluate curve

To determine the local coordinates offset from the curve we need: normal vector of each point
with this we can move along the normal vector a desired distance to determine boundaries and mesh points

In [ ]:
nurbs.operations.normal(curve)
trackWidth = 70 #pixels
meshRes = 10 #number of points per row
scale = trackWidth/meshRes
for i in range(meshRes):
    mesh_row = []
      for point in curve:
          new_pt = scale*curve.normal.normalisedVector + point
          mesh_row.append(new_pt)
    mesh.append(mesh_row)


# Caculate curvature

In [ ]:
numerator = dotproduct(curve.velocity, curve.acceleration)
denomerator = curve.velocity ^ 3
k  = numerator/ denomerator

optimize pseudo above to use matrix multiplications rather than looping over each point individually

In [ ]:
radius = 1/k

# Calculating time
time taken to traverse the track along that specific spline

first we need to create a sample spline, to do this
select a random point from each row of the mesh and generate a spline from it, then calculate the curvature to find the 'radius'

with a sample spline generated with know curvature we can apply the physic formulas